In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!git clone https://github.com/anandbhoraskar/img2vec.git
!cd img2vec && git pull origin master

fatal: destination path 'img2vec' already exists and is not an empty directory.
From https://github.com/anandbhoraskar/img2vec
 * branch            master     -> FETCH_HEAD
Already up-to-date.


In [4]:
# !unzip /content/drive/My\ Drive/Sem1/10701/data/FID-300.zip 
# !unzip /content/drive/My\ Drive/Sem1/10701/data/CSFID.zip 

In [5]:
data_path = "../../data/FID-300/"

In [6]:
import sys
import os
sys.path.insert(1, 'img2vec')
from img2vec_pytorch import Img2Vec
import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)

from PIL import Image
import numpy as np

In [7]:
from scipy.io import loadmat
label_table = loadmat(os.path.join(data_path, 'label_table.mat'))
label_table = label_table['label_table']
# print(label_table['label_table'])

In [8]:
references = np.arange(1, 1176)
# print(references)

test_val = np.arange(1, 301)
# print(test_val)

In [9]:
def file_path(i:int, data:str = 'ref'):
  if data == 'ref':
    # print('ref')
    return data_path+"/references/"+"{:05d}".format(i)+".png"
  else:
    # print('test')
    return data_path+"/tracks_cropped/"+"{:05d}".format(i)+".jpg"

In [10]:
print(file_path(1, 'ref'))
print(file_path(1, 'test'))

../../data/FID-300//references/00001.png
../../data/FID-300//tracks_cropped/00001.jpg


In [11]:
ref_img_list = []
for i in references:
  img = Image.open(file_path(i,'ref'))
  img = img.convert("RGB")
  ref_img_list.append(img)

In [12]:
test_img_list = []
for i in test_val:
  img = Image.open(file_path(i,'test'))
  img = img.convert("RGB")
  test_img_list.append(img)

In [13]:
import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)
img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=512)

In [14]:
# img2vec.layer_output_size = 2048
# print(img2vec.layer_output_size)

In [15]:
ref_vec_list = []
test_vec_list = []

ref_vec_list = [img2vec.get_vec(img, tensor=True) for img in ref_img_list]
test_vec_list = [img2vec.get_vec(img, tensor=True) for img in test_img_list]

In [16]:
ref_vec_list_np = np.array([np.array(vec) for vec in ref_vec_list])
test_vec_list_np = np.array([np.array(vec) for vec in test_vec_list])

In [17]:
ref_vec_list_np = ref_vec_list_np.reshape(1175, 2048)
test_vec_list_np = test_vec_list_np.reshape(300, 2048)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
score = cosine_similarity(test_vec_list_np, ref_vec_list_np)

In [19]:
score_sort = (-score).argsort(1)

In [20]:
pos_array = []
for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
  pos_array.append(np.where(a==c)[0][0])
pos_array = np.array(pos_array)
  

In [21]:
t = 5
thresh = t*pos_array.shape[0]/100
acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
print("top {}: {}".format(t, acc))

top 5: 22.666666666666668


In [22]:
t = 10
thresh = t*pos_array.shape[0]/100
acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
print("top {}: {}".format(t, acc))

top 10: 32.666666666666664


In [23]:
def find_scores(img2vec, layer_output_size=512, return_tensor=False):
  if return_tensor:
    ref_vec_list = np.array([img2vec.get_vec(img).mean(1).flatten().numpy() for img in ref_img_list])
    test_vec_list = np.array([img2vec.get_vec(img).mean(1).flatten().numpy() for img in test_img_list])
  else:
    ref_vec_list = np.array([img2vec.get_vec(img) for img in ref_img_list])
    test_vec_list = np.array([img2vec.get_vec(img) for img in test_img_list])

  ref_vec_list = ref_vec_list.reshape(1175, layer_output_size)
  test_vec_list = test_vec_list.reshape(300, layer_output_size)

  # test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
  # ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]
  score = cosine_similarity(test_vec_list, ref_vec_list)
  score_sort = (-score).argsort(1)

  pos_array = []
  for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
    pos_array.append(np.where(a==c)[0][0])
  pos_array = np.array(pos_array)

  t = 5
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

  t = 10
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))
  return score

In [24]:
def mcncc(v1, v2):
  den = np.linalg.norm(v1, axis=1)*np.linalg.norm(v2, axis=1) + 1e-12
  num = (v1*v2).sum(axis=1)
  return (num/den).sum()

# def mcncc_all(test_vec_list, ref_vec_list):
#     den = np.linalg.norm(test_vec_list, axis=2)*np.linalg.norm(v2, axis=2) + 1e-12
#     num = (v1*v2).sum(axis=1)
    

def find_scores_mcncc(img2vec, layer_output_size=512, return_tensor=False):
  ref_vec_list = np.array([img2vec.get_vec(img).numpy() for img in ref_img_list])
  test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])

  ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
  test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])

  test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
  ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]
    
  print("computing score")
#   score = mcncc_all(test_vec_list, ref_vec_list)
  score = np.zeros((test_vec_list.shape[0], ref_vec_list.shape[0]))
  for i, v1 in enumerate(test_vec_list):
    print(i)
    for j, v2 in enumerate(ref_vec_list):
      score[i, j] = mcncc(v1, v2)

  score_sort = (-score).argsort(1)

  pos_array = []
  for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
    pos_array.append(np.where(a==c)[0][0])
  pos_array = np.array(pos_array)

  t = 5
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

  t = 10
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))
  return score

In [25]:
# # Best performing
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer3', layer_output_size=256, 
#     channels=14, return_embedding=True)
# score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

## Cosine distance

In [26]:
# img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=2048, centre_crop=True)
# print(img2vec.layer_output_size)
# score_layer4 = find_scores(img2vec)

# # Cos sim (centre crop)
# # top 5: 24.666666666666668
# # top 10: 35.0


# # Normalize:
# # top 5: 22.333333333333332
# # top 10: 30.333333333333332

In [27]:
# img2vec_layer3 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer3', layer_output_size=1024, centre_crop=True)
# score_layer3 = find_scores(img2vec_layer3)

# # Cos
# # top 5: 6.0
# # top 10: 8.666666666666666

In [28]:
# img2vec_layer2 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer2', layer_output_size=512, centre_crop=True)
# score_layer2 = find_scores(img2vec_layer2)

# # Cos
# # top 5: 7.0
# # top 10: 11.0

In [29]:
# img2vec_layer1 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer1', layer_output_size=256, centre_crop=True)
# score_layer1 = find_scores(img2vec_layer1)

# #Cos
# # top 5: 5.333333333333333
# # top 10: 7.0

## Top crop

In [30]:
# img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=2048, top_crop=True)
# print(img2vec.layer_output_size)
# score_layer4 = find_scores(img2vec, 2048)


In [31]:
# img2vec_layer3 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer3', layer_output_size=1024, top_crop=True)
# score_layer3 = find_scores(img2vec_layer3, 1024)

In [32]:
# img2vec_layer2 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer2', layer_output_size=512, top_crop=True)
# score_layer2 = find_scores(img2vec_layer2, 512)

In [33]:
# img2vec_layer1 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer1', layer_output_size=256, top_crop=True)
# score_layer1 = find_scores(img2vec_layer1, 256)

## No crop

In [34]:
# img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=2048)
# print(img2vec.layer_output_size)
# score_layer4 = find_scores(img2vec, 2048)


In [35]:
# img2vec_layer3 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer3', layer_output_size=1024)
# score_layer3 = find_scores(img2vec_layer3, 1024)

In [36]:
# img2vec_layer2 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer2', layer_output_size=512)
# score_layer2 = find_scores(img2vec_layer2, 512)

In [37]:
# img2vec_layer1 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer1', layer_output_size=256)
# score_layer1 = find_scores(img2vec_layer1, 256)

In [38]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer1', layer_output_size=256, 
#     channels=56, return_embedding=True)
# score_layer2_conv = find_scores(img2vec, 3136, return_tensor=True)

In [39]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer2', layer_output_size=128, 
#     channels=28, return_embedding=True)
# score_layer2_conv = find_scores(img2vec, 784, return_tensor=True)

In [40]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer3', layer_output_size=256, 
#     channels=14, return_embedding=True)
# score_layer2_conv = find_scores(img2vec, 196, return_tensor=True)

In [41]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer4', layer_output_size=512, 
#     channels=7, return_embedding=True)
# score_layer2_conv = find_scores(img2vec, 49, return_tensor=True)

In [42]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer1', layer_output_size=64, 
#     channels=56, return_embedding=True, centre_crop=True)
# score_layer2_conv = find_scores(img2vec, 3136, return_tensor=True)

In [43]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer2', layer_output_size=128, 
#     channels=28, return_embedding=True, centre_crop=True)
# score_layer2_conv = find_scores(img2vec, 784, return_tensor=True)

In [44]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer3', layer_output_size=256, 
#     channels=14, return_embedding=True, centre_crop=True)
# score_layer2_conv = find_scores(img2vec, 196, return_tensor=True)

In [45]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer4', layer_output_size=512, 
#     channels=7, return_embedding=True, centre_crop=True)
# score_layer2_conv = find_scores(img2vec, 49, return_tensor=True)

## MCNCC

In [46]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer4', layer_output_size=256, 
#     channels=14, return_embedding=True)
# score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [47]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer3', layer_output_size=256, 
#     channels=14, return_embedding=True)
# score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [48]:
# img2vec = img2vec_pytorch.Img2Vec(
#     cuda=True, layer='layer2', layer_output_size=256, 
#     channels=14, return_embedding=True)
# score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer1', layer_output_size=256, 
    channels=14, return_embedding=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

computing score
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


## MCNCC (Centre crop)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer4', layer_output_size=256, 
    channels=14, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer2', layer_output_size=256, 
    channels=14, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer1', layer_output_size=256, 
    channels=14, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

## MCNCC (Top crop)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer4', layer_output_size=256, 
    channels=14, return_embedding=True, top_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True, top_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer2', layer_output_size=256, 
    channels=14, return_embedding=True, top_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer1', layer_output_size=256, 
    channels=14, return_embedding=True, top_crop=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

layer_id = 5
model = models.resnet50(pretrained=True)
modules = list(model.children())[:layer_id] 
net_base = nn.Sequential(*modules)

In [ ]:
modules

In [ ]:
list(model.children())

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True)

 

In [ ]:
a = img2vec.get_vec(ref_img_list[0]).numpy()
# test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])

# ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
# test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])


In [ ]:
ref_vec_list = np.array([img2vec.get_vec(img).numpy() for img in ref_img_list])
test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])

In [ ]:
ref_vec_list.shape

In [ ]:
ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])

In [ ]:
ref_vec_list.shape

In [ ]:
test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]

In [ ]:
img = ref_img_list[0]

In [ ]:
img.shape

In [ ]:
img2vec.scaler(img2vec.to_tensor(img))

In [ ]:
img2vec.to_tensor(img).shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
width, height = img.size
img.crop((0, 0, width, width))

In [ ]:
img.shape

In [ ]:
a = np.random.rand(2,2,2,2)

In [ ]:
a

In [ ]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True)

In [ ]:
ref_vec_list = np.array([img2vec.get_vec(img).numpy() for img in ref_img_list])
test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])
ref_vec_list.shape

In [ ]:
ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])
ref_vec_list.shape

In [ ]:
test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]

In [ ]:
ref_vec_list.shape

In [ ]:
ref_vec_list@test_vec_list.T

In [ ]:
np.zeros((2,2,2,2))[:, None].shape

In [ ]:
test_vec_list.mean(2)[:, None].shape

In [ ]:
test_vec_list-test_vec_list.mean(2)[:, None]

In [ ]:
import torch
torch.tensor([1,2,3.0]).norm(axis=0)